In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset_1 = dataset.copy()    # make a copy of original data
dataset_1.loc[:, 'sex'] = dataset_1.loc[:, 'sex'].map({'female' : -1, 'male' : 1})    # integer encoding of sex
dataset_1.loc[:, 'smoker'] = dataset_1.loc[:, 'smoker'].map({'no' : -1, 'yes' : 1})    # integer encoding of smoker
dataset_1.loc[:, 'vertical'] = dataset_1.loc[:, 'region'].map({'northeast' : 1, 'northwest' : 1, 'southeast' : -1, 'southwest' : -1})    # treating north, south as +1, -1
dataset_1.loc[:, 'horizontal'] = dataset_1.loc[:, 'region'].map({'northeast' : 1, 'northwest' : -1, 'southeast' : 1, 'southwest' : -1})    # treating east, west as +1, -1
dataset_1.pop('region')    # removing region column
dataset_1 = dataset_1.astype(np.float32)    # convert all data to float32

In [ ]:
train_dataset = dataset_1.sample(frac = 0.8)    # randomly select ~80% of data for training
test_dataset = dataset_1.drop(train_dataset.index)    # remaining data for testing
train_labels = train_dataset.pop('expenses')    # expenses column will be target column
test_labels = test_dataset.pop('expenses')

In [ ]:
normalizer = tf.keras.layers.Normalization(axis = -1)    # normalization layer
normalizer.adapt(train_dataset)    # normalizes on all the columns of train_dataset, though it's unnecessary for integer-encoded columns
model = keras.Sequential([normalizer,
                          layers.Dense(1)])    # adding more layers while keeping activation None (to keep the model linear) can give better results
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 10),
              loss = keras.losses.MeanAbsoluteError(),
              metrics = ['mae', 'mse'])    # this makes output of evaluate loss, mae, mse

In [ ]:
history = model.fit(train_dataset,
                    train_labels,
                    epochs = 150,    # increasing learning_rate can reduce this but how far can we go?
                    verbose = 0)

In [ ]:
fig, axs = plt.subplots()
axs.semilogy(history.history['loss'])
axs.axhline(3500, color = 'darkgreen', linewidth = 1)
axs.set_xlabel('epoch')
axs.set_ylabel('loss')
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
